In [1]:
from Bio import SeqIO
import click
import pandas as pd
import sys
sys.path.append("../")
from src.config import ROOT_DIR
from os.path import join

import tqdm

Project Directory: /data2/mito_lineage


In [2]:
ref_fa = join(ROOT_DIR, "data/processed/genomes/refdata-cellranger-GRCh38-3.0.0/MT.fasta")
out_f = join(ROOT_DIR, "data/processed/amplicons/MT_amplicons.bed")

In [3]:
chromosome="MT" 
amplicon_length=150 
pcr_length=(18,22)


In [4]:
record_dict = SeqIO.to_dict(SeqIO.parse(ref_fa, "fasta"))
try:
    seq = record_dict[chromosome]
except KeyError:
    print(f"{chromosome} not found in {ref_fa}. Please check if you have the proper name")
    
print(seq)

ID: MT
Name: MT
Description: MT dna:chromosome chromosome:GRCh38:MT:1:16569:1 REF
Number of features: 0
Seq('GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGG...ATG', SingleLetterAlphabet())


In [5]:
positions = range(0,len(seq), 150)

### Create bed file of the MT regions broken up into approximate pcr sizes

In [13]:
bed_df = pd.DataFrame(columns=["chrom", "chromStart", "chromEnd", "name", "score", "strand"])

forward_sequences = dict()


#forward_sequences = SeqIO.fas
for ind in tqdm.tqdm(range(50,len(seq)-100,amplicon_length)):
    bed_df = pd.concat((bed_df, pd.DataFrame({"chrom":chromosome,
                                    "chromStart":ind, "chromEnd":min(ind+amplicon_length, len(seq)-2), "name":ind, "score":0, "strand":"+"},
                                           index=[ind])))
    forward_sequences[f"{ind}-{ind+100+amplicon_length}"] = str(seq.seq[ind:min(ind+500, len(seq))])

    
# reverse_sequences
reverse_sequences = dict()
rev_seq = seq.reverse_complement()
for ind in tqdm.tqdm(range(50,len(rev_seq)-100,amplicon_length)):
    reverse_sequences[f"{ind}-{ind+100+amplicon_length}"] = str(rev_seq.seq[ind:min(ind+500, len(rev_seq))])

    
reverse_sequences

100%|██████████| 110/110 [00:00<00:00, 132388.36it/s]


{'50-300': 'ACCCTGAAGTAGGAACCAGATGTCGGATACAGTTCACTTTAGCTACCCCCAAGTGTTATGGGCCCGGAGCGAGGAGAGTAGCACTCTTGTGCGGGATATTGATTTCACGGAGGATGGTGGTCAAGGGACCCCTATCTGAGGGGGGTCATCCATGGGGACGAGAAGGGATTTGACTGTAATGTGCTATGTACGGTAAATGGCTTTATGTACTATGTACTGTTAAGGGTGGGTAGGTTTGTTGGTATCCTAGTGGGTGAGGGGTGGCTTTGGAGTTGCAGTTGATGTGTGATAGTTGAGGGTTGATTGCTGTACTTGCTTGTAAGCATGGGGAGGGGGTTTTGATGTGGATTGGGTTTTTATGTACTACAGGTGGTCAAGTATTTATGGTACCGTACAATATTCATGGTGGCTGGCAGTAATGTACGAAATACATAGCGGTTGTTGATGGGTGAGTCAATACTTGGGTGGTACCCAAATCTGCTTCCCCATGAAAGAACAGA',
 '200-450': 'CATGGGGACGAGAAGGGATTTGACTGTAATGTGCTATGTACGGTAAATGGCTTTATGTACTATGTACTGTTAAGGGTGGGTAGGTTTGTTGGTATCCTAGTGGGTGAGGGGTGGCTTTGGAGTTGCAGTTGATGTGTGATAGTTGAGGGTTGATTGCTGTACTTGCTTGTAAGCATGGGGAGGGGGTTTTGATGTGGATTGGGTTTTTATGTACTACAGGTGGTCAAGTATTTATGGTACCGTACAATATTCATGGTGGCTGGCAGTAATGTACGAAATACATAGCGGTTGTTGATGGGTGAGTCAATACTTGGGTGGTACCCAAATCTGCTTCCCCATGAAAGAACAGAGAATAGTTTAAATTAGAATCTTAGCTTTGGGTGCTAATGGTGGAGTTAAAGACTTTTTCTCTGATTTGTCCTTGGAAAAAGGTTTTCATCTCCGGTTTACAAGACTGGTGTATTAGTTTATA

In [14]:
forward_sequences

{'50-300': 'TTGGTATTTTCGTCTGGGGGGTATGCACGCGATAGCATTGCGAGACGCTGGAGCCGGAGCACCCTATGTCGCAGTATCTGTCTTTGATTCCTGCCTCATCCTATTATTTATCGCACCTACGTTCAATATTACAGGCGAACATACTTACTAAAGTGTGTTAATTAATTAATGCTTGTAGGACATAATAATAACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACAAAAAATTTCCACCAAACCCCCCCTCCCCCGCTTCTGGCCACAGCACTTAAACACATCTCTGCCAAACCCCAAAAACAAAGAACCCTAACACCAGCCTAACCAGATTTCAAATTTTATCTTTTGGCGGTATGCACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCA',
 '200-450': 'AAGTGTGTTAATTAATTAATGCTTGTAGGACATAATAATAACAATTGAATGTCTGCACAGCCACTTTCCACACAGACATCATAACAAAAAATTTCCACCAAACCCCCCCTCCCCCGCTTCTGGCCACAGCACTTAAACACATCTCTGCCAAACCCCAAAAACAAAGAACCCTAACACCAGCCTAACCAGATTTCAAATTTTATCTTTTGGCGGTATGCACTTTTAACAGTCACCCCCCAACTAACACATTATTTTCCCCTCCCACTCCCATACTACTAATCTCATCAATACAACCCCCGCCCATCCTACCCAGCACACACACACCGCTGCTAACCCCATACCCCGAACCAACCAAACCCCAAAGACACCCCCCACAGTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTCACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTA